# [**Broutonlab**](https://broutonlab.com/) face recognition with masks pipeline

## [**github repo**](https://github.com/broutonlab/face-id-with-medical-masks) with solution

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Check GPU resources</font></b>
!nvidia-smi

Wed May 13 21:14:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Import requirements</font></b>
import os
import sys
import cv2
from matplotlib import pyplot as plt
import sys
import numpy as np

import torch
from torch import nn
from tqdm.notebook import tqdm

from torch.utils.data import DataLoader

%matplotlib inline

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Clone and build face-alignment git repo</font></b>
!git clone https://github.com/1adrianb/face-alignment
%cd face-alignment
!pip install -r requirements.txt
!python setup.py install
import face_alignment

Cloning into 'face-alignment'...
remote: Enumerating objects: 563, done.
remote: Total 563 (delta 0), reused 0 (delta 0), pack-reused 563
Receiving objects: 100% (563/563), 3.51 MiB | 2.53 MiB/s, done.
Resolving deltas: 100% (340/340), done.
/content/face-alignment
running install
running bdist_egg
running egg_info
creating face_alignment.egg-info
writing face_alignment.egg-info/PKG-INFO
writing dependency_links to face_alignment.egg-info/dependency_links.txt
writing requirements to face_alignment.egg-info/requires.txt
writing top-level names to face_alignment.egg-info/top_level.txt
writing manifest file 'face_alignment.egg-info/SOURCES.txt'
writing manifest file 'face_alignment.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/face_alignment
copying face_alignment/api.py -> build/lib/face_alignment
copying face_alignment/__init__.py -> build/lib/face_alignment
copying 

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Download and import face mask SDK</font></b>
!git clone https://github.com/broutonlab/face-id-with-medical-masks.git
%cd face-id-with-medical-masks

from masked_face_sdk.mask_generation_utils import end2end_mask_generation
from masked_face_sdk.pipeline_dataset_loader import PipelineFacesDatasetGenerator

from masked_face_sdk.pipeline_dataset_loader \
    import PipelineFacesDatasetGenerator
from masked_face_sdk.neural_network_modules \
    import Backbone, ArcFaceLayer, FaceRecognitionModel, resnet18
from masked_face_sdk.training_utils import default_acc_function, test_embedding_net




Cloning into 'face-id-with-medical-masks'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 283 (delta 1), reused 0 (delta 0), pack-reused 279
Receiving objects: 100% (283/283), 40.87 MiB | 9.17 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/face-alignment/face-id-with-medical-masks


In [ ]:
!gdown --id 1b64prOr4_E8gcD1Q_cVZkFnSzNVfGwU_
!unzip face_recognition_with_masks_dataset.zip
!ls
# Pathes to datasets for face recognition in Keras-like format
root_train_dataset_path = 'test_large/'
root_test_dataset_path = 'test_small/'


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Generate masks database</font></b>
# Generate masks database
!python3 generate_masks_database.py \
    --masks-folder=data/masked_faces/ \
    --database-file=data/masks_base.json \
    --verbose --skip-warnings

Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/checkpoints/s3fd-619a316812.pth
100% 85.7M/85.7M [00:10<00:00, 8.84MB/s]
Downloading: "https://www.adrianbulat.com/downloads/python-fan/2DFAN4-11f355bf06.pth.tar" to /root/.cache/torch/checkpoints/2DFAN4-11f355bf06.pth.tar
100% 91.2M/91.2M [00:11<00:00, 8.49MB/s]
  2% 4/244 [00:56<59:13, 14.80s/it]

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Prepare training dataset</font></b>
# Prepare training dataset

!python3 apply_masks_to_face_recognition_dataset.py \
    --face-dataset-folder={root_train_dataset_path} \
    --masks-database=data/masks_base.json \
    --verbose \
    --use-cuda

  0%|                                                   | 0/101 [00:00<?, ?it/s]Warning: No faces were detected.
Skipping and delete from base follow image: /media/alexey/DataDisk/datasets/vggface2_with_face_boxes_markup/test/n000001/0301_01.jpg, because can't find faces landmarks
 56%|███████████████████████▋                  | 57/101 [41:55<35:26, 48.32s/it]Warning: No faces were detected.
Skipping and delete from base follow image: /media/alexey/DataDisk/datasets/vggface2_with_face_boxes_markup/test/n001291/0445_01.jpg, because can't find faces landmarks
100%|███████████████████████████████████████| 101/101 [1:05:58<00:00, 39.19s/it]


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Prepare test dataset</font></b>
# Prepare test dataset

!python3 apply_masks_to_face_recognition_dataset.py \
    --face-dataset-folder={root_test_dataset_path} \
    --masks-database=data/masks_base.json \
    --verbose \
    --use-cuda

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Initialize constants</font></b>
# Init constants

batch_size = 100
n_jobs = 4
epochs = 3000
image_shape = (112, 112)
embedding_size = 256
device = 'cuda:0'

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Initialize base variables for training</font></b>
# Init base variables for training

generator_train_dataset = PipelineFacesDatasetGenerator(
    root_train_dataset_path,
    image_shape
)

train_loader = DataLoader(
        generator_train_dataset,
        batch_size=batch_size,
        num_workers=n_jobs,
        shuffle=True,
        drop_last=True
)

model = FaceRecognitionModel(
    backbone=Backbone(
        backbone=resnet18(pretrained=True),
        embedding_size=embedding_size,
        input_shape=(3, image_shape[0], image_shape[1])
    ),
    head=ArcFaceLayer(
        embedding_size=embedding_size,
        num_classes=generator_train_dataset.num_classes
    )
)
model = model.to(device)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.00001)

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Run test for embedding net</font></b>
print(
    'Start accuracy rate = {:.5f}'.format(
        test_embedding_net(root_test_dataset_path, image_shape, model, device)
    )
)

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Perform training process</font></b>
# Training process

epoch_loss = []
epoch_test_acc = []

for epoch in range(1, epochs + 1):
    model.train()

    batches_count = len(train_loader)
    avg_epoch_loss = 0
    avg_epoch_acc = 0
    
    with tqdm(total=batches_count) as pbar:
        for i, (_img, _y_true) in enumerate(train_loader):
            img = _img.to(device)
            y_true = _y_true.to(device)

            optimizer.zero_grad()
            y_pred = model(img, y_true)

            loss = loss_function(
                y_pred,
                y_true
            )

            loss.backward()

            optimizer.step()

            acc = default_acc_function(
                y_pred,
                torch.nn.functional.one_hot(
                    y_true,
                    num_classes=y_pred.size(-1)
                ).to(y_pred.dtype).to(device)
            ).numpy()

            pbar.postfix = \
                'Epoch: {}/{}, loss: {:.8f}, ' \
                'avg acc: {:.8f}'.format(
                    epoch,
                    epochs,
                    loss.item(),
                    acc
                )

            avg_epoch_loss += \
                loss.item() / y_true.size(0) / batches_count

            avg_epoch_acc += acc / batches_count
            

            pbar.update(1)
        
    test_acc = test_embedding_net(root_test_dataset_path, image_shape, model, device)
    
    print('Test accuracy rate: {:.5f}'.format(test_acc))
    
    epoch_loss.append(avg_epoch_loss)
    epoch_test_acc.append(test_acc)

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Plot the results</font></b>
plt.figure(figsize=(8, 8))
plt.title('Train loss per epoch')
plt.xlabel('Epoch number')
plt.ylabel('Binary crossentropy value')
plt.plot(list(range(1, len(epoch_loss) + 1)), epoch_loss)


plt.figure(figsize=(8, 8))
plt.title('Test accuracy rate per epoch')
plt.xlabel('Epoch number')
plt.ylabel('Accuracy rate')
plt.plot(list(range(1, len(epoch_test_acc) + 1)), epoch_test_acc, color='orange')

plt.show()